##### SQL 삽입 (p.15)

1) DB-API 사용 예제

In [ ]:
from django.shortcuts import render

def update_board(request):
    ...
	with dbconn.cursor() as curs:
		# 외부로부터 입력받은 값을 검증 없이 사용할 경우 안전하지 않다.
		name = request.POST.get('name', '') ## !
		content_id = request.POST.get('content_id', '') ## !

		# 사용자의 검증되지 않은 입력으로 부터 동적으로 쿼리문 생성
		sql_query = "update board set name='" + name + "' where content_id='“ + content_id + "'"
		
		# 외부 입력값이 검증 없이 쿼리로 수행되어 안전하지 않다.
		curs.execute(sql_query) ## !
		curs.commit()
		return render(request, '/success.html')

2) ORM 사용 예제

In [ ]:
from django.shortcuts import render
from app.models import Member

def member_search(request):
    # 외부로부터 입력 값을 가져온다.
    name = request.POST.get('name', '') ## !
    
    # 외부로부터 입력 받은 값을 검증 없이 쿼리문 생성에 사용하여
    # 안전하지 않다.
    query=“select * from member where name=‘” + name + “’”
    
    # 외부 입력 값을 검증 없이 사용한 쿼리문을 raw()함수로 실행하면
    # 안전하지 않다.
    data = Member.objects.raw(query) ## !
    return render(request, '/member_list.html', {'member_list':data})

##### 코드삽입 (p.20)

1) eval()함수 사용 예제

In [ ]:
from django.shortcuts import render

def route(request):
    #외부로 입력받은 값을 검증 없이 사용하면 안전하지 않다.
    message = request.POST.get('message', '') ## !
    
    # eval함수에 외부 입력값을 검증 없이 사용할 경우 Python 코드가
    # 실행될 수 있어 위험하다.
    
    ret = eval(message) ## !
    return render(request, '/success.html', {'data':ret})

2) exec()함수 사용 예제

In [ ]:
from django.shortcuts import render

def request_rest_api(request):
    function_name = request.POST.get('function_name', '') ## !
    
    # 사용자에게 전달받은 함수명을 검증하지 않고 실행
    # 입력 값으로 “import plaform \n platform.system()” 등을 입력 시
    # 시스템정보 노출 위험이 있다.
    ret = exec('{}()'.format(function_name)) ## !
    
    return render(request, '/success', {'data':ret})

##### 경로 조작 및 자원 삽입 (p.24)

1) 경로 조작 예제

In [ ]:
import os
from django.shortcuts import render

def get_info(request):
    # 외부로부터 파일명을 입력받고 있다.
    request_file = request.POST.get('request_file') ## !
    filename, file_ext = os.path.splitext(request_file)
    file_ext = file_ext.lower()
    
    if file_ext not in ['.txt', '.csv']:
        return render(request, '/error.html', {'error':'파일을 열수 없습니다.'})
    
    # 외부로부터 입력받은 값을 검증 없이 파일 처리에 사용하였다.
    with open(request_file) as f: ## !
        data = f.read()
    
    return render(request, '/success.html', {'data':data})

2) 자원 삽입 예제

In [ ]:
import socket
from django.shortcuts import render

def get_info(request):
    port = request.POST.get('port') ## !
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        # 외부로부터 입력받은 검증되지 않은 포트번호를 이용하여
        # 소켓을 바인딩 하여 사용하고 있어 안전하지 않다.
        s.bind(('127.0.0.1', port)) ## ! port
        ...
        return render(request, '/success')
    return render(request, '/error', {'error':'소켓연결 실패'})

##### 크로스사이트 스크립트(XSS) (p.29)

1) Django 예제

In [ ]:
from django.shortcuts import render
from django.utils.safestring import mark_safe ## !

def profile_link(request):
    # 외부 입력값을 검증 없이 HTML 태그 생성의 인자로 사용
    profile_url = request.POST.get('profile_url')
    profile_name = requst.POST.get('profile_name')
    
    object_link = '<a href="{}">{}</a>'.format(profile_url, profile_name)
    # mark_safe함수는 Django의 XSS escape 정책을 따르지 않는다.
    object_link = mark_safe(object_link) ## !
    
    return render(request, 'my_profile.html',{'object_link':object_link})

In [ ]:
<!doctype html>
<html>
    <body>
        <div class="content">
            {% autoescape off %} ## !
                // autoescape off로 해당 블록내의 데이터는 XSS 공격에
                // 노출될 수 있음.
                {{ content }} ## !
            {% endautoescape %} ## !
        </div>
        <div class="content2">
            //safe 필터 사용으로 XSS 공격에 노출될 수 있음.
            {{ content | safe }} ## !
        </div>
    </body>
</html>

2) Flak 예제

In [ ]:
from flask import Flask, request, render_template

@app.route('/search', methods=['POST'])
def search():
    search_keyword = request.form.get('search_keyword') ## !
    # 사용자의 입력을 아무런 검증 또는 치환 없이 동적 웹 페이지에 사용하고 있어
    # 크로스 사이트 스크립트가 발생할 수 있다
    return render_template('search.html', search_keyword=search_keyword) ## !

##### 운영체제 명령어 삽입 (p.34)

In [ ]:
import os
from django.shortcuts import render

def execute_command(request):
    app_name_string = request.POST.get('app_name','') ## !
    # 입력받은 파라미터를 제한하지 않아 외부 입력값으로 전달된
    # 모든 프로그램이 실행될 수 있음
    os.system(app_name_string) ## !
    return render(request, '/success.html')

In [ ]:
import subprocess
from django.shortcuts import render

def execute_command(request):
    date = request.POST.get('date','') ## !
    # 입력받은 파라미터를 제한하지 않아 전달된 모든 프로그램이
    # 실행될 수 있음
    cmd_str = “cmd /c backuplog.bat ” + date
    subprocess.run(cmd_str, shell=True) ## !
    return render(request, '/success.html')

##### 위험한 형식 파일 업로드 (p.38)

In [ ]:
from django.shortcuts import render
from django.core.files.storage import FileSystemStorage

def file_upload(request):
    if request.FILES['upload_file']:
        # 사용자로부터 업로드 되는 파일에 대해 검증 없이 저장하고 있어
        # 안전하지 않다.
        upload_file = request.FILES['upload_file'] ## !
        fs = FileSystemStorage(location='media/screenshot', base_url='media/screenshot')
        # 업로드 하는 파일에 대한 크기, 개수, 확장자 등을 검증 하지 않음.
        filename = fs.save(upload_file.name, upload_file)
        return render(request, '/success.html', {'filename':filename})
    return render(request, '/error.html', {'error':'파일 업로드 실패'})

##### 신뢰되지 않은 URL주소로 자동접속 연결 (p.41)

In [ ]:
from django.shortcuts import redirect

def redirect_url(request):
    url_string = request.POST.get('url', '') ## !
    # 사용자 입력에 포함된 URL 주소로 리다이렉트 하는 경우
    # 피싱 사이트로 접속되는 등 사용자가 피싱 공격에 노출될 수 있다
    return redirect(url_string) ## !

##### 부적절한 XML 외부 개체 참조 (p.44)

In [ ]:
from xml.sax import make_parser
from xml.sax.handler import feature_external_ges
from xml.dom.pulldom import parseString, START_ELEMENT
from django.shortcuts import render
from .model import comments

def get_xml(request):
    if request.method == “GET”:
        data = comments.objects.all()
        com = data[0].comment ## !
        return render(request, '/xml_view.html', {'com':com}) ## !
    
    elif request.method == “POST”:
        parser = make_parser()
        parser.setFeature(feature_external_ges, True) ## !
        doc = parseString(request.body.decode(‘utf-8’), parser=parser)
        for event, node in doc:
            if event == START_ELEMENT and node.tagName == “foo”:
                doc.expandNode(node)
                text = node.toxml()
        comments.objects.filter(id=1).update(comment=text);
        return render(request, '/xml_view.html')

##### XML 삽입 (p.46)

In [ ]:
from lxml import etree

def parse_xml(request):
    user_name = request.POST.get('user_name', '') ## !
    
    parser = etree.XMLParser(resolve_entities=False)
    tree = etree.parse('user.xml', parser)
    root = tree.getroot()
    
    # 검증되지 않은 외부 입력값 user_name를 사용하여
    # 안전하지 않은 질의문이 작성되에 query변수에 저장
    path = "/collection/users/user[@name='" + user_name + "']/home/text()" ## !
    elmts = root.xpath(query)
    return render(request, 'parse_xml.html', {'xml_element':elmts})

##### LDAP 삽입 (p.49)

In [ ]:
from ldap3 import Connection, Server, ALL
from django.shortcuts import render

def ldap_query(request):
    # 외부 입력값으로 LDAP 쿼리문의 인자에 검증 없이 사용하면
    # 안전하지 않다.
    search_keyword = request.POST.get('search_keyword','') ## !
    
    dn = server.config['bind_dn']
    password = server.config['password']
    
    address = 'ldap.badSoruce.com'
    server = Server(address, get_info=ALL)
    conn = Connection(server, user=dn, password, auto_bind=True )
    
    # 사용자 입력을 필터링 하지 않는 경우 공격자의 권한 상승으로
    # 이어질 수 있다
    search_str = '(&(objectclass=%s))' % search_keyword ## !
    
    conn.search('dc=company,dc=com', search_str, attributes=['sn', 'cn', 'address', 'mail', 'mobile', 'uid']) ## ! search_str
    return render(request, '/ldap_query_response.html', {'ldap':conn.entries})

#####  크로스사이트 요청 위조(CSRF) (p.52)

1. Django 프레임워크 사용 - 미들웨어 설정(settings.py) 사례

In [ ]:
MIDDLEWARE = [
    'django.contrib.sessions.middleware.SessionMiddleware',
    # MIDDLEWARE 목록에서 csrf 항목을 삭제 또는 주석처리 하면
    # Django 앱에서 csrf 유효성 검사가 전역적으로 제거 됨
    # 'django.middleware.csrf.CsrfViewMiddleware', ## !
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.locale.LocaleMiddleware',
    ......
]

2. Django 프레임워크 사용 - 뷰 기능 설정(views.py) 사례

In [ ]:
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

# csrf.exempt 데코레이터로 미들웨어에서 보호되는 CSRF 기능을 해제한다.
@csrf.exempt ## !
def pay_to_point(request):
    user_id = request.POST.get('user_id', '')
    pay = request.POST.get('pay', '')
    product_info = request.POST.get('product_info', '')
    
    ret = pay(user_id, pay, product_info)
    
    return render(request, '/view_wallet.html', {'wallet':ret})

3. Django 프레임워크 사용 - 템플릿 설정 사례

In [ ]:
<!--html page-->
<form action="" method="POST">
<!-- form 태그 내부에 csrf_token 미적용--> <! >
    <table>
        {{form.as_table}}
    </table>
    <input type="submit"/>
</form>

4. Flask 프레임워크 사용 - app 설정 사례

In [ ]:
from flask import Flask

app = Flask(__name__)

5. Flask 프레임워크 사용 - 템플릿 설정 사례

In [ ]:
<form action="" method="POST">
<!-- form 태그 내부에 csrf_token 미적용--> <! >
    <table>
        {{as_table}}
    </table>
    <input type="submit"/>
</form>

##### 서버사이드 요청 위조 (p.58)

In [ ]:
from django.shortcuts import render
import requests

def call_third_party_api(request):
    addr = request.POST.get('address', '') ## !
    
    # 사용자가 입력한 주소를 검증하지 않고 HTTP 요청을 보내고 응답을
    # 사용자에게 반환
    result = requests.get(addr).text ## !
    return render(request, '/result.html', {'result':result})

##### HTTP 응답분할 (p.61)

In [ ]:
from django.http import HttpResponse

def route(request):
    content_type = request.POST.get('contnet-type')
    # 외부 입력값을 검증 또는 필터링 하지 않고
    # 응답헤더의 값으로 포함시켜 회신한다.
    ......
    res = HttpResponse()
    res['Content-Type'] = content_type ## !
    return res

##### 보안기능 결정에 사용되는 부적절한 입력값 (p.63)

In [ ]:
from django.shortcuts import render

def init_password(request):
    # 쿠키에서 권한 정보를 가져옴
    roll = request.COOKIE['roll'] ## !
    request_id = request.POST.get('user_id', '')
    request_mail = request.POST.get('user_email','')
    # 쿠키에서 가져온 권한이 관리자인지 비교
    if roll == 'admin' ## !
        # 사용자의 패스워드 초기화 및 메일 발송 처리
        password_init_and_sendmail(request_id, request_mail)
        return render(request, '/sucess.html')
    else:
        return render(request, '/failed.html')

##### 포맷 스트링 삽입 (p.66)

In [ ]:
from django.shortcuts import render
AUTHENTICATE_KEY = 'Passw0rd' ## !

def make_user_message(request):
    user_info = get_user_info(request.POST.get('user_id', ''))
    
    format_string = request.POST.get('msg_format', '') ## !
    # 내부의 민감한 정보가 외부로 노출될 수 있다.
    # 사용자가 입력한 문자열을 포맷 문자열로 사용하고 있어 안전하지 않다
    message = format_string.format(user=user_info) ## !
    
    return render(request, '/user_page', {'message':message})